In [1]:
! pip install pytket-quantinuum

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 90 kB 4.6 MB/s 
     |████████████████████████████████| 13.4 MB 22.3 MB/s 
     |████████████████████████████████| 106 kB 73.5 MB/s 
     |████████████████████████████████| 4.0 MB 63.4 MB/s 
     |████████████████████████████████| 133 kB 56.7 MB/s 
     |████████████████████████████████| 47 kB 4.1 MB/s 
     |████████████████████████████████| 103 kB 48.1 MB/s 
  Attempting uninstall: jinja2
    Found existing installation: Jinja2 2.11.3
    Uninstalling Jinja2-2.11.3:
      Successfully uninstalled Jinja2-2.11.3
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
noteb

Submitting to Quantinuum Emulators via pytket
This notebook contains examples for running quantum circuits on Quantinuum's emulators via pytket.

An emulator can be used to get an idea of what a quantum device will output for our quantum circuit. Emulators differ from simulators in that they model the physical and noise model of the device whereas simulators may model noise parameters, but not physical parameters. The Quantinuum emulators run on a physical noise model of the Quantinuum H-Series devices. There are various noise/error parameters modeled. For detailed information on the noise model, see the Quantinuum System Model H1 Emulator Product Data Sheet on the user portal or at Quantinuum H-series.


In [2]:
from pytket.circuit import Circuit
from pytket.circuit.display import render_circuit_jupyter

Set up Bell State

In [3]:
circuit = Circuit(2, name="Bell Test")
circuit.H(0)
circuit.CX(0, 1)
circuit.measure_all()

[H q[0]; CX q[0], q[1]; Measure q[0] --> c[0]; Measure q[1] --> c[1]; ]

In [4]:
render_circuit_jupyter(circuit)

In [5]:
from pytket.extensions.quantinuum import QuantinuumBackend

In [9]:
machine = "H1-1E"
backend = QuantinuumBackend(device_name=machine)
backend.login()

Enter your Quantinuum email: manu.chaudhary@ku.edu
Enter your Quantinuum password: ··········


In [10]:
print(machine, "status:", backend.device_state(device_name=machine))

H1-1E status: online


Compile the circuit to the Quantinuum backend with `get_compiled_circuit`. See the pytket User Manual for more information on all the options that are available.

In [11]:
compiled_circuit = backend.get_compiled_circuit(circuit, optimisation_level=1)

In [12]:
render_circuit_jupyter(compiled_circuit)

Check the circuit HQC cost before running on the emulator.

In [13]:
n_shots = 100
backend.cost(compiled_circuit, n_shots=n_shots, syntax_checker="H1-1SC")

5.66

Run the circuit on the emulator chosen.

In [14]:
handle = backend.process_circuit(compiled_circuit, n_shots=n_shots)
print(handle)

('1ee49fa7f6eb4a5593d48d3f7ab08f93', 'null')


Check the job status.

In [15]:
status = backend.circuit_status(handle)
print(status)

CircuitStatus(status=<StatusEnum.COMPLETED: 'Circuit has completed. Results are ready.'>, message='{"name": "Bell Test", "submit-date": "2022-12-22T04:55:45.141188", "result-date": "2022-12-22T04:56:12.145873", "queue-position": null, "cost": "5.66", "error": null}', error_detail=None, completed_time=None, queued_time=None, submitted_time=None, running_time=None, cancelled_time=None, error_time=None, queue_position=None)


Once a job's status returns completed, return results with the get_result function.

In [16]:
result = backend.get_result(handle)

In [17]:
result

BackendResult(q_bits={},c_bits={c[1]: 0, c[0]: 1},counts=None,shots=[[  0]
 [  0]
 [192]
 [  0]
 [  0]
 [192]
 [  0]
 [192]
 [192]
 [192]
 [192]
 [192]
 [  0]
 [  0]
 [  0]
 [  0]
 [192]
 [192]
 [  0]
 [  0]
 [  0]
 [192]
 [192]
 [  0]
 [192]
 [192]
 [  0]
 [  0]
 [  0]
 [  0]
 [192]
 [192]
 [192]
 [  0]
 [  0]
 [  0]
 [192]
 [192]
 [192]
 [192]
 [192]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [192]
 [  0]
 [192]
 [192]
 [ 64]
 [  0]
 [192]
 [  0]
 [192]
 [192]
 [192]
 [  0]
 [  0]
 [192]
 [192]
 [192]
 [  0]
 [  0]
 [192]
 [192]
 [  0]
 [192]
 [  0]
 [  0]
 [192]
 [  0]
 [192]
 [192]
 [192]
 [  0]
 [192]
 [  0]
 [192]
 [  0]
 [192]
 [192]
 [192]
 [192]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [192]
 [  0]
 [192]
 [  0]
 [192]
 [  0]
 [  0]
 [192]
 [  0]
 [  0]],state=None,unitary=None,density_matrix=None)

In [18]:
import json

In [19]:
with open("pytket_emulator_example.json", "w") as file:
    json.dump(result.to_dict(), file)

In [20]:
result = backend.get_result(handle)
print(result.get_distribution())

{(0, 0): 0.51, (1, 0): 0.01, (1, 1): 0.48}


In [21]:
print(result.get_counts())

Counter({(0, 0): 51, (1, 1): 48, (1, 0): 1})


## **Noiseless Emulation**

In [22]:
n_shots = 100
no_error_model_handle = backend.process_circuit(
    compiled_circuit, n_shots=n_shots, noisy_simulation=False
)
print(no_error_model_handle)

('1b3db9c9d33c4d6cbf858dd36e59df8d', 'null')


In [25]:
no_error_model_status = backend.circuit_status(no_error_model_handle)
print(no_error_model_status)

CircuitStatus(status=<StatusEnum.COMPLETED: 'Circuit has completed. Results are ready.'>, message='{"name": "Bell Test", "submit-date": "2022-12-22T05:05:56.943838", "result-date": "2022-12-22T05:06:00.927972", "queue-position": null, "cost": "5.66", "error": null}', error_detail=None, completed_time=None, queued_time=None, submitted_time=None, running_time=None, cancelled_time=None, error_time=None, queue_position=None)


In [26]:
no_error_model_result = backend.get_result(no_error_model_handle)

In [27]:
no_error_model_result

BackendResult(q_bits={},c_bits={c[1]: 0, c[0]: 1},counts=None,shots=[[192]
 [192]
 [  0]
 [  0]
 [  0]
 [192]
 [192]
 [192]
 [192]
 [  0]
 [  0]
 [192]
 [192]
 [  0]
 [192]
 [  0]
 [192]
 [192]
 [192]
 [192]
 [192]
 [192]
 [192]
 [192]
 [192]
 [192]
 [192]
 [  0]
 [192]
 [  0]
 [  0]
 [192]
 [192]
 [  0]
 [192]
 [192]
 [192]
 [  0]
 [  0]
 [192]
 [192]
 [  0]
 [192]
 [  0]
 [  0]
 [  0]
 [192]
 [  0]
 [192]
 [192]
 [  0]
 [192]
 [  0]
 [  0]
 [192]
 [192]
 [192]
 [  0]
 [  0]
 [192]
 [  0]
 [  0]
 [192]
 [  0]
 [  0]
 [192]
 [192]
 [  0]
 [192]
 [  0]
 [192]
 [  0]
 [192]
 [192]
 [192]
 [  0]
 [192]
 [192]
 [  0]
 [  0]
 [  0]
 [192]
 [192]
 [  0]
 [192]
 [  0]
 [  0]
 [192]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [192]
 [192]
 [  0]
 [192]
 [  0]
 [192]],state=None,unitary=None,density_matrix=None)

In [28]:
with open("pytket_emulator_noiseless_example.json", "w") as file:
    json.dump(result.to_dict(), file)

In [29]:
no_error_model_result = backend.get_result(no_error_model_handle)
print(no_error_model_result.get_distribution())

{(0, 0): 0.45, (1, 1): 0.55}


In [30]:
print(no_error_model_result.get_counts())

Counter({(1, 1): 55, (0, 0): 45})


## **Noise Parameters**

The emulator runs with default error parameters that represent a noise environment similar to the physical devices. 
The error-params option can be used to override these error parameters and do finer-grain tweaks of the error model.
For detailed information on the noise model, see the Quantinuum System Model H1 Emulator Product Data Sheet on the user portal or or Quantinuum H-Series page or the Quantinuum Application Programming Interface (API) on the user portal.

In this section, examples are given for experimenting with the noise and error parameters of the emulators. These are advanced options and not recommended to start with when doing initial experiments.

Note: All the noise parameters are used together any time a simulation is run. If only some of the parameters are specified, the rest of the parameters are used at their default settings. The parameters to override are specified with the options parameter.



*   Physical Noise
*   Dephasing Noise
*   Arbitrary Angle Noise Scaling
*   Scaling

## Physical Noise

In [31]:
handle = backend.process_circuit(
    compiled_circuit,
    n_shots=100,
    request_options={
        "options": {
            "error-params": {
                "p1": 4e-5,
                "p2": 3e-3,
                "p_meas": 3e-3,
                "p_init": 4e-5,
                "p_crosstalk_meas": 1e-5,
                "p_crosstalk_init": 3e-5,
                "p1_emission": 6e-6,
                "p2_emission": 2e-4,
            }
        }
    },
)

In [32]:
result = backend.get_result(handle)

In [33]:
print(result.get_distribution())

{(0, 0): 0.49, (1, 1): 0.51}


## Dephasing Noise

In [34]:
handle = backend.process_circuit(
    compiled_circuit,
    n_shots=100,
    request_options={
        "options": {
            "error-params": {
                "coherent_dephasing_rate": 0.2,
                "incoherent_dephasing_rate": 0.3,
                "coherent_dephasing": False,  # False => run the incoherent noise model
                "transport_dephasing": False,  # False => turn off transport dephasing error
                "idle_dephasing": False,  # False => turn off idel dephasing error
            },
        }
    },
)

In [35]:
result = backend.get_result(handle)

In [36]:
print(result.get_distribution())

{(0, 0): 0.46, (1, 1): 0.54}


Arbitrary Angle Noise Scaling

In [37]:
handle = backend.process_circuit(
    compiled_circuit,
    n_shots=100,
    request_options={
        "options": {
            "error-params": {
                "przz_a": 1.09,
                "przz_b": 0.051,
                "przz_c": 1.365,
                "przz_d": 0.035,
            },
        }
    },
)

In [38]:
result = backend.get_result(handle)

In [39]:
print(result.get_distribution())

{(0, 0): 0.57, (0, 1): 0.01, (1, 0): 0.01, (1, 1): 0.41}


Scaling

In [40]:
handle = backend.process_circuit(
    compiled_circuit,
    n_shots=100,
    request_options={
        "options": {
            "error-params": {
                "scale": 0.1,  # scale error rates linearly by 0.1
            },
        }
    },
)

In [41]:
result = backend.get_result(handle)

In [42]:
print(result.get_distribution())

{(0, 0): 0.44, (1, 1): 0.56}


In [43]:
handle = backend.process_circuit(
    compiled_circuit,
    n_shots=100,
    request_options={
        "options": {
            "error-params": {
                "p1_scale": 0.1,
                "p2_scale": 0.1,
                "meas_scale": 0.1,
                "init_scale": 0.1,
                "memory_scale": 0.1,
                "emission_scale": 0.1,
                "crosstalk_scale": 0.1,
                "leakage_scale": 0.1,
            },
        }
    },
)

In [44]:
result = backend.get_result(handle)

In [45]:
print(result.get_distribution())

{(0, 0): 0.49, (1, 1): 0.51}


## **Stabilizer Emulator**

By default, emulations are run using a state-vector emulator, which simulates any quantum operation. However, if the quantum operations are all Clifford gates, it can be faster for complex circuits to use the stabilizer emulator. The stabilizer emulator is requested in the setup of the QuantinuumBackend with the simulator input option. This only applies to Quantinuum emulators.

In [47]:
machine = "H1-1E"

In [48]:
stabilizer_backend = QuantinuumBackend(device_name=machine, simulator="stabilizer")

In [49]:
print(machine, "status:", stabilizer_backend.device_state(device_name=machine))
print("Simulation type:", stabilizer_backend.simulator_type)

H1-1E status: online
Simulation type: stabilizer


In [50]:
n_shots = 100
stabilizer_handle = stabilizer_backend.process_circuit(
    compiled_circuit, n_shots=n_shots
)
print(stabilizer_handle)

('99b6225b91e54cba8494c05284da04f9', 'null')


In [51]:
stabilizer_status = stabilizer_backend.circuit_status(stabilizer_handle)
print(stabilizer_status)

CircuitStatus(status=<StatusEnum.QUEUED: 'Circuit is queued.'>, message='{"name": "Bell Test", "submit-date": "2022-12-22T05:35:51.053493", "result-date": null, "queue-position": null, "cost": "5.66", "error": null}', error_detail=None, completed_time=None, queued_time=None, submitted_time=None, running_time=None, cancelled_time=None, error_time=None, queue_position=None)


In [52]:
stabilizer_result = stabilizer_backend.get_result(stabilizer_handle)

In [53]:
stabilizer_result

BackendResult(q_bits={},c_bits={c[1]: 0, c[0]: 1},counts=None,shots=[[  0]
 [192]
 [  0]
 [  0]
 [192]
 [  0]
 [  0]
 [192]
 [  0]
 [192]
 [  0]
 [192]
 [  0]
 [  0]
 [192]
 [  0]
 [  0]
 [192]
 [  0]
 [192]
 [192]
 [  0]
 [192]
 [  0]
 [  0]
 [  0]
 [192]
 [192]
 [192]
 [  0]
 [192]
 [192]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [192]
 [  0]
 [  0]
 [128]
 [  0]
 [192]
 [192]
 [  0]
 [  0]
 [192]
 [192]
 [  0]
 [192]
 [  0]
 [  0]
 [192]
 [  0]
 [  0]
 [  0]
 [  0]
 [192]
 [192]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [192]
 [192]
 [192]
 [  0]
 [192]
 [192]
 [192]
 [192]
 [  0]
 [192]
 [  0]
 [192]
 [192]
 [  0]
 [  0]
 [  0]
 [  0]
 [192]
 [192]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [192]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [192]
 [192]],state=None,unitary=None,density_matrix=None)

In [54]:
with open("pytket_emulator_stabilizer_example.json", "w") as file:
    json.dump(result.to_dict(), file)

In [55]:
stabilizer_result = stabilizer_backend.get_result(stabilizer_handle)
print(stabilizer_result.get_distribution())

{(0, 0): 0.6, (0, 1): 0.01, (1, 1): 0.39}


In [56]:
print(stabilizer_result.get_counts())

Counter({(0, 0): 60, (1, 1): 39, (0, 1): 1})


## **Noiseless Stabilizer**

In [57]:
handle = backend.process_circuit(
    compiled_circuit,
    n_shots=100,
    request_options={
        "options": {
            "simulator": "stabilizer",
            "error-model": False,
        }
    },
)

In [58]:
result = backend.get_result(handle)

In [59]:
print(result.get_distribution())

{(0, 0): 0.56, (1, 1): 0.44}
